In [12]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import nltk


nltk.download('stopwords')
train = pd.read_csv("/kaggle/input/bagofwordsmeto/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

print("训练数据形状:", train.shape)
print(train.head())

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


训练数据形状: (25000, 3)
         id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3  "3630_4"          0  "It must be assumed that those who praised thi...
4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...


In [18]:
def review_to_words(raw_review):
 
    #去除HTML标签
    review_text = BeautifulSoup(raw_review, "html.parser").get_text()
    
    #保留字母字符
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    #转换为小写并分割成单词
    words = letters_only.lower().split()
    
    #将停用词转换为集合(提高查找速度)
    stops = set(stopwords.words("english"))
    
    #去除停用词
    meaningful_words = [w for w in words if w not in stops]
    
    #将单词列表合并为一个字符串返回
    return " ".join(meaningful_words)

In [14]:
#影评数量
num_reviews = train["review"].size

#初始化一个空列表来存储清洗后的影评
clean_train_reviews = []

print("开始清洗和解析训练集中的影评...")

#处理每条影评
for i in range(num_reviews):
    if (i+1) % 1000 == 0:
        print(f"已处理 {i+1} 条影评，共 {num_reviews} 条")
    #清洗函数
    clean_train_reviews.append(review_to_words(train["review"][i]))

print("\n清洗完成!")

开始清洗和解析训练集中的影评...
已处理 1000 条影评，共 25000 条
已处理 2000 条影评，共 25000 条
已处理 3000 条影评，共 25000 条
已处理 4000 条影评，共 25000 条
已处理 5000 条影评，共 25000 条
已处理 6000 条影评，共 25000 条
已处理 7000 条影评，共 25000 条
已处理 8000 条影评，共 25000 条
已处理 9000 条影评，共 25000 条
已处理 10000 条影评，共 25000 条
已处理 11000 条影评，共 25000 条
已处理 12000 条影评，共 25000 条
已处理 13000 条影评，共 25000 条
已处理 14000 条影评，共 25000 条
已处理 15000 条影评，共 25000 条
已处理 16000 条影评，共 25000 条
已处理 17000 条影评，共 25000 条
已处理 18000 条影评，共 25000 条
已处理 19000 条影评，共 25000 条
已处理 20000 条影评，共 25000 条
已处理 21000 条影评，共 25000 条
已处理 22000 条影评，共 25000 条
已处理 23000 条影评，共 25000 条
已处理 24000 条影评，共 25000 条
已处理 25000 条影评，共 25000 条

清洗完成!


In [15]:
print("创建词袋特征...")

vectorizer = CountVectorizer(
    analyzer="word",
    tokenizer=None,
    preprocessor=None,
    stop_words=None,
    max_features=5000 
)

#拟合模型并转换训练数据
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()
print("特征矩阵形状:", train_data_features.shape)
vocab = vectorizer.get_feature_names_out()
print("词汇表前10个词:", vocab[:10])

#统计每个词出现次数
dist = np.sum(train_data_features, axis=0)
for tag, count in zip(vocab[:10], dist[:10]):
    print(count, tag)

创建词袋特征...
特征矩阵形状: (25000, 5000)
词汇表前10个词: ['abandoned' 'abc' 'abilities' 'ability' 'able' 'abraham' 'absence'
 'absent' 'absolute' 'absolutely']
187 abandoned
125 abc
108 abilities
454 ability
1259 able
85 abraham
116 absence
83 absent
352 absolute
1485 absolutely


In [16]:
print("训练随机森林模型...")

#初始化
forest = RandomForestClassifier(n_estimators=100)

#拟合模型
forest = forest.fit(train_data_features, train["sentiment"])

print("模型训练完成!")

训练随机森林模型...
模型训练完成!


In [17]:
test = pd.read_csv("/kaggle/input/bagofwordsmeto/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

#清洗
print("清洗和解析测试集影评...")
num_reviews = len(test["review"])
clean_test_reviews = []

for i in range(num_reviews):
    if (i+1) % 1000 == 0:
        print(f"已处理 {i+1} 条影评，共 {num_reviews} 条")
    clean_review = review_to_words(test["review"][i])
    clean_test_reviews.append(clean_review)

# 将测试数据转换为词袋特征
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

#预测
result = forest.predict(test_data_features)

#提交文件
output = pd.DataFrame(data={"id": test["id"], "sentiment": result})
output.to_csv("Bag_of_Words_model.csv", index=False, quoting=3)

print("预测结果已保存到Bag_of_Words_model.csv")

清洗和解析测试集影评...
已处理 1000 条影评，共 25000 条
已处理 2000 条影评，共 25000 条
已处理 3000 条影评，共 25000 条
已处理 4000 条影评，共 25000 条
已处理 5000 条影评，共 25000 条
已处理 6000 条影评，共 25000 条
已处理 7000 条影评，共 25000 条
已处理 8000 条影评，共 25000 条
已处理 9000 条影评，共 25000 条
已处理 10000 条影评，共 25000 条
已处理 11000 条影评，共 25000 条
已处理 12000 条影评，共 25000 条
已处理 13000 条影评，共 25000 条
已处理 14000 条影评，共 25000 条
已处理 15000 条影评，共 25000 条
已处理 16000 条影评，共 25000 条
已处理 17000 条影评，共 25000 条
已处理 18000 条影评，共 25000 条
已处理 19000 条影评，共 25000 条
已处理 20000 条影评，共 25000 条
已处理 21000 条影评，共 25000 条
已处理 22000 条影评，共 25000 条
已处理 23000 条影评，共 25000 条
已处理 24000 条影评，共 25000 条
已处理 25000 条影评，共 25000 条
预测结果已保存到Bag_of_Words_model.csv
